<a href="https://colab.research.google.com/github/yagizKanbur/Predicting-Mood-Using-Song-Features/blob/master/Happy_Relaxed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1.Introduction to SVM**

info

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.over_sampling import SMOTE, SVMSMOTE

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


**Making Labels**

In [ ]:
data = pd.read_csv('deam/static_annotations_averaged_songs_1_2000.csv')

valenceMean = data[[' valence_mean']]
arousalMean = data[[' arousal_mean']]
valence = np.asarray(valenceMean)
arousal = np.asarray(arousalMean)
data["label"] = ""

for i in range(0,len(valence)):
  if arousal[i] > 4.5 and valence[i] > 4.5:
    data["label"][i] = "1"
    #data["label"][i] = "Happy"
  elif arousal[i] <= 4.5 and valence[i] > 4.5:
    data["label"][i] = "1"#Relaxed
    #data["label"][i] = "Relaxed"
  elif arousal[i] <= 4.5 and valence[i] <= 4.5:
    data["label"][i] = "0"#Sad
    #data["label"][i] = "Sad"
  else: 
    data["label"][i] = "0"#Annoyed
    #data["label"][i] = "Annoyed"


data.to_csv("deam/static_annotations_averaged_songs_1_2000.csv", index=False)







/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launc

In [ ]:
data = pd.read_csv('deam/static_annotations_averaged_songs_2000_2058.csv')

valenceMean = data[[' valence_mean']]
arousalMean = data[[' arousal_mean']]
valence = np.asarray(valenceMean)
arousal = np.asarray(arousalMean)
data["label"] = ""


for i in range(0,len(valence)):
  if arousal[i] > 4.5 and valence[i] > 4.5:
    data["label"][i] = "1"
    #data["label"][i] = "Happy"
  elif arousal[i] <= 4.5 and valence[i] > 4.5:
    data["label"][i] = "1"
    #data["label"][i] = "Relaxed"
  elif arousal[i] <= 4.5 and valence[i] <= 4.5:
    data["label"][i] = "0"
    #data["label"][i] = "Sad"
  else: 
    data["label"][i] = "0"
    #data["label"][i] = "Annoyed"



data.to_csv("deam/static_annotations_averaged_songs_2000_2058.csv", index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykerne

**Extracted Features**

In [ ]:
ef = pd.read_csv('deam/Audio Features.csv')

data = pd.read_csv('deam/static_annotations_averaged_songs_1_2000.csv')
arousalLabel = data[[ 'label']]
aL = np.asarray(arousalLabel)

lateData =pd.read_csv('deam/static_annotations_averaged_songs_2000_2058.csv')
lateLabel = lateData[[ 'label']]
lL = np.asarray(lateLabel)

ef["Label"] = ""

Label =  np.concatenate((aL, lL))

for i in range (0,1746):
  ef["Label"][i] = Label[i][0]

ef.to_csv("deam/featureTrain.csv", index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [ ]:
data = pd.read_csv('bestFeatures.csv')
column = np.asarray(data)
print(len(column[0]))
print(len(column))

**Feature Selection**

In [ ]:
import pandas as pd
import numpy as np
names = ["un","zcr_mean","energy_mean","energy_entropy_mean","spectral_centroid_mean","spectral_spread_mean","spectral_entropy_mean","spectral_flux_mean","spectral_rolloff_mean","mfcc_1_mean","mfcc_2_mean","mfcc_3_mean","mfcc_4_mean","mfcc_5_mean","mfcc_6_mean","mfcc_7_mean","mfcc_8_mean","mfcc_9_mean","mfcc_10_mean","mfcc_11_mean","mfcc_12_mean","mfcc_13_mean","chroma_1_mean","chroma_2_mean","chroma_3_mean","chroma_4_mean","chroma_5_mean","chroma_6_mean","chroma_7_mean","chroma_8_mean","chroma_9_mean","chroma_10_mean","chroma_11_mean","chroma_12_mean","chroma_std_mean","delta zcr_mean","delta energy_mean","delta energy_entropy_mean","delta spectral_centroid_mean","delta spectral_spread_mean","delta spectral_entropy_mean","delta spectral_flux_mean","delta spectral_rolloff_mean","delta mfcc_1_mean","delta mfcc_2_mean","delta mfcc_3_mean","delta mfcc_4_mean","delta mfcc_5_mean","delta mfcc_6_mean","delta mfcc_7_mean","delta mfcc_8_mean","delta mfcc_9_mean","delta mfcc_10_mean","delta mfcc_11_mean","delta mfcc_12_mean","delta mfcc_13_mean","delta chroma_1_mean","delta chroma_2_mean","delta chroma_3_mean","delta chroma_4_mean","delta chroma_5_mean","delta chroma_6_mean","delta chroma_7_mean","delta chroma_8_mean","delta chroma_9_mean","delta chroma_10_mean","delta chroma_11_mean","delta chroma_12_mean","delta chroma_std_mean","zcr_std","energy_std","energy_entropy_std","spectral_centroid_std","spectral_spread_std","spectral_entropy_std","spectral_flux_std","spectral_rolloff_std","mfcc_1_std","mfcc_2_std","mfcc_3_std","mfcc_4_std","mfcc_5_std","mfcc_6_std","mfcc_7_std","mfcc_8_std","mfcc_9_std","mfcc_10_std","mfcc_11_std","mfcc_12_std","mfcc_13_std","chroma_1_std","chroma_2_std","chroma_3_std","chroma_4_std","chroma_5_std","chroma_6_std","chroma_7_std","chroma_8_std","chroma_9_std","chroma_10_std","chroma_11_std","chroma_12_std","chroma_std_std","delta zcr_std","delta energy_std","delta energy_entropy_std","delta spectral_centroid_std","delta spectral_spread_std","delta spectral_entropy_std","delta spectral_flux_std","delta spectral_rolloff_std","delta mfcc_1_std","delta mfcc_2_std","delta mfcc_3_std","delta mfcc_4_std","delta mfcc_5_std","delta mfcc_6_std","delta mfcc_7_std","delta mfcc_8_std","delta mfcc_9_std","delta mfcc_10_std","delta mfcc_11_std","delta mfcc_12_std","delta mfcc_13_std","delta chroma_1_std","delta chroma_2_std","delta chroma_3_std","delta chroma_4_std","delta chroma_5_std","delta chroma_6_std","delta chroma_7_std","delta chroma_8_std","delta chroma_9_std","delta chroma_10_std","delta chroma_11_std","delta chroma_12_std","delta chroma_std_std","Label"]
df_features = pd.read_csv('featureSelection.csv', names = names)
df_features.drop("un", axis=1)
df_features.head()

X = df_features[["zcr_mean","energy_mean","energy_entropy_mean","spectral_centroid_mean","spectral_spread_mean","spectral_entropy_mean","spectral_flux_mean","spectral_rolloff_mean","mfcc_1_mean","mfcc_2_mean","mfcc_3_mean","mfcc_4_mean","mfcc_5_mean","mfcc_6_mean","mfcc_7_mean","mfcc_8_mean","mfcc_9_mean","mfcc_10_mean","mfcc_11_mean","mfcc_12_mean","mfcc_13_mean","chroma_1_mean","chroma_2_mean","chroma_3_mean","chroma_4_mean","chroma_5_mean","chroma_6_mean","chroma_7_mean","chroma_8_mean","chroma_9_mean","chroma_10_mean","chroma_11_mean","chroma_12_mean","chroma_std_mean","delta zcr_mean","delta energy_mean","delta energy_entropy_mean","delta spectral_centroid_mean","delta spectral_spread_mean","delta spectral_entropy_mean","delta spectral_flux_mean","delta spectral_rolloff_mean","delta mfcc_1_mean","delta mfcc_2_mean","delta mfcc_3_mean","delta mfcc_4_mean","delta mfcc_5_mean","delta mfcc_6_mean","delta mfcc_7_mean","delta mfcc_8_mean","delta mfcc_9_mean","delta mfcc_10_mean","delta mfcc_11_mean","delta mfcc_12_mean","delta mfcc_13_mean","delta chroma_1_mean","delta chroma_2_mean","delta chroma_3_mean","delta chroma_4_mean","delta chroma_5_mean","delta chroma_6_mean","delta chroma_7_mean","delta chroma_8_mean","delta chroma_9_mean","delta chroma_10_mean","delta chroma_11_mean","delta chroma_12_mean","delta chroma_std_mean","zcr_std","energy_std","energy_entropy_std","spectral_centroid_std","spectral_spread_std","spectral_entropy_std","spectral_flux_std","spectral_rolloff_std","mfcc_1_std","mfcc_2_std","mfcc_3_std","mfcc_4_std","mfcc_5_std","mfcc_6_std","mfcc_7_std","mfcc_8_std","mfcc_9_std","mfcc_10_std","mfcc_11_std","mfcc_12_std","mfcc_13_std","chroma_1_std","chroma_2_std","chroma_3_std","chroma_4_std","chroma_5_std","chroma_6_std","chroma_7_std","chroma_8_std","chroma_9_std","chroma_10_std","chroma_11_std","chroma_12_std","chroma_std_std","delta zcr_std","delta energy_std","delta energy_entropy_std","delta spectral_centroid_std","delta spectral_spread_std","delta spectral_entropy_std","delta spectral_flux_std","delta spectral_rolloff_std","delta mfcc_1_std","delta mfcc_2_std","delta mfcc_3_std","delta mfcc_4_std","delta mfcc_5_std","delta mfcc_6_std","delta mfcc_7_std","delta mfcc_8_std","delta mfcc_9_std","delta mfcc_10_std","delta mfcc_11_std","delta mfcc_12_std","delta mfcc_13_std","delta chroma_1_std","delta chroma_2_std","delta chroma_3_std","delta chroma_4_std","delta chroma_5_std","delta chroma_6_std","delta chroma_7_std","delta chroma_8_std","delta chroma_9_std","delta chroma_10_std","delta chroma_11_std","delta chroma_12_std","delta chroma_std_std"]]
Y = df_features[["Label"]]

X_train = np.asarray(X)

from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.ensemble import RandomForestClassifier

# Estimator
estimator = RandomForestClassifier(n_estimators=10, n_jobs=4)
# Step Forward Feature Selector
StepBackward = sfs(estimator,k_features="best",forward=False,floating=False,verbose=2,scoring='accuracy',cv=5)
StepBackward.fit(X_train,Y.values.ravel())

StepBackward.subsets_
StepBackward.k_feature_names_
StepBackward.k_score_

FileNotFoundError: ignored

In [ ]:
StepBackward.subsets_



In [ ]:
StepBackward.k_feature_names_

In [ ]:
StepBackward.k_score_

In [ ]:
data = pd.read_csv('deam/featureTrain.csv')
best_features = data[["energy_mean","energy_entropy_mean","spectral_rolloff_mean","mfcc_1_mean","mfcc_2_mean","mfcc_4_mean","chroma_2_mean","chroma_4_mean","chroma_9_mean","delta energy_entropy_mean","delta spectral_centroid_mean","delta spectral_flux_mean","delta mfcc_11_mean","delta chroma_1_mean","delta chroma_4_mean","delta chroma_7_mean","delta chroma_11_mean","spectral_centroid_std","mfcc_2_std","mfcc_5_std","mfcc_10_std","mfcc_12_std","chroma_8_std","chroma_9_std","chroma_10_std","chroma_11_std","delta spectral_centroid_std","delta spectral_entropy_std","delta mfcc_13_std","delta chroma_1_std","delta chroma_2_std","delta chroma_3_std","delta chroma_7_std","delta chroma_8_std","delta chroma_12_std","Label"]]

best_features.to_csv('bestFeatures.csv', index = False)


**SVM**

In [ ]:
data = pd.read_csv('bestFeatures.csv')

training_set, test_set = train_test_split(data, test_size = 0.2)

X_train = training_set.iloc[:, [0,34]].values
y_train = training_set.iloc[:,35].values
X_test = test_set.iloc[:, [0,34]].values
y_test = test_set.iloc[:,35].values

smote = SMOTE()
X_train, y_train = smote.fit_sample(X_train, y_train)
#X_test, y_test = smote.fit_sample(X_test, y_test)

from sklearn.svm import SVC
classifier = SVC(kernel='rbf') #Kernels: linear, poly, sigmoid, rbf
classifier.fit(X_train,y_train)

y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

print("Normal")
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

# Create confusion matrix
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])

Normal
[[ 91  51]
 [ 78 129]]
              precision    recall  f1-score   support

           0       0.54      0.64      0.59       142
           1       0.72      0.62      0.67       207

    accuracy                           0.63       349
   macro avg       0.63      0.63      0.63       349
weighted avg       0.64      0.63      0.63       349

0.6303724928366762


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Predicted,0,1
Actual,,
0,91,51
1,78,129


**KNN**

In [ ]:
dataset = pd.read_csv('bestFeatures.csv')

training_set, test_set = train_test_split(dataset, test_size = 0.2)

X_train = training_set.iloc[:, [0,34]].values
y_train = training_set.iloc[:,35].values
X_test = test_set.iloc[:, [0,34]].values
y_test = test_set.iloc[:,35].values

smote = SMOTE()
X_train, y_train = smote.fit_sample(X_train, y_train)
#X_test, y_test = smote.fit_sample(X_test, y_test)

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 17)#17
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

# Create confusion matrix
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])

0.6418338108882522
[[ 94  53]
 [ 72 130]]
              precision    recall  f1-score   support

           0       0.57      0.64      0.60       147
           1       0.71      0.64      0.68       202

    accuracy                           0.64       349
   macro avg       0.64      0.64      0.64       349
weighted avg       0.65      0.64      0.64       349



/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Predicted,0,1
Actual,,
0,94,53
1,72,130


**Random Forest Reggression**

In [ ]:
dataset = pd.read_csv('bestFeatures.csv')

training_set, test_set = train_test_split(dataset, test_size = 0.2, random_state = 1)

X_train = training_set.iloc[:, [0,34]].values
y_train = training_set.iloc[:,35].values
X_test = test_set.iloc[:, [0,34]].values
y_test = test_set.iloc[:,35].values


sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=10, random_state=1)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.4151862464183381
Mean Squared Error: 0.26151862464183384
Root Mean Squared Error: 0.5113889172066929


**Random Forest**

In [ ]:
dataset = pd.read_csv('bestFeatures.csv')
training_set, test_set = train_test_split(dataset, test_size = 0.2, random_state = 1)

X_train = training_set.iloc[:, [0,34]].values
y_train = training_set.iloc[:,35].values
X_test = test_set.iloc[:, [0,34]].values
y_test = test_set.iloc[:,35].values

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

smote = SMOTE()
X_train, y_train = smote.fit_sample(X_train, y_train)
#X_test, y_test = smote.fit_sample(X_test, y_test)


from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=20, n_jobs = 7)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))


# Create confusion matrix
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.6217765042979942
[[ 84  55]
 [ 77 133]]
              precision    recall  f1-score   support

           0       0.52      0.60      0.56       139
           1       0.71      0.63      0.67       210

    accuracy                           0.62       349
   macro avg       0.61      0.62      0.61       349
weighted avg       0.63      0.62      0.63       349



Predicted,0,1
Actual,,
0,84,55
1,77,133


**MLP Reggression**

In [ ]:
dataset = pd.read_csv('bestFeatures.csv')
training_set, test_set = train_test_split(dataset, test_size = 0.2, random_state = 1)

X_train = training_set.iloc[:, [0,34]].values
y_train = training_set.iloc[:,35].values
X_test = test_set.iloc[:, [0,34]].values
y_test = test_set.iloc[:,35].values

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

from sklearn.neural_network import MLPRegressor
reg = MLPRegressor(hidden_layer_sizes=(64,64,64),activation="relu" ,random_state=1, max_iter=2000).fit(X_train, y_train)
y_pred=reg.predict(X_test)
from sklearn.metrics import r2_score
print("The Score with " + str((r2_score(y_pred, y_test))))

The Score with -4.778001891672382


**MLP Classification**

In [ ]:
dataset = pd.read_csv('bestFeatures.csv')
training_set, test_set = train_test_split(dataset, test_size = 0.2,random_state=1)

X_train = training_set.iloc[:, [0,34]].values
y_train = training_set.iloc[:,35].values
X_test = test_set.iloc[:, [0,34]].values
y_test = test_set.iloc[:,35].values

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


#smote = SMOTE()
#X_train, y_train = smote.fit_sample(X_train, y_train)
#X_test, y_test = smote.fit_sample(X_test, y_test)

from sklearn.neural_network import MLPClassifier

classifier = MLPClassifier(hidden_layer_sizes=(256,128,64,32),activation="relu",random_state=1).fit(X_train, y_train)
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

# Create confusion matrix
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])

[[ 63  76]
 [ 27 183]]
              precision    recall  f1-score   support

           0       0.70      0.45      0.55       139
           1       0.71      0.87      0.78       210

    accuracy                           0.70       349
   macro avg       0.70      0.66      0.67       349
weighted avg       0.70      0.70      0.69       349

0.7048710601719198


Predicted,0,1
Actual,,
0,63,76
1,27,183


**Naive Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB
dataset = pd.read_csv('bestFeatures.csv')

training_set, test_set = train_test_split(dataset, test_size = 0.2)

X_train = training_set.iloc[:, [0,34]].values
y_train = training_set.iloc[:,35].values
X_test = test_set.iloc[:, [0,34]].values
y_test = test_set.iloc[:,35].values

smote = SMOTE()
X_train, y_train = smote.fit_sample(X_train, y_train)

classifier = GaussianNB()
#_pred = classifier.fit(X_train, y_train).predict(X_test)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))


# Create confusion matrix
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])

[[ 99  47]
 [ 88 115]]
              precision    recall  f1-score   support

           0       0.53      0.68      0.59       146
           1       0.71      0.57      0.63       203

    accuracy                           0.61       349
   macro avg       0.62      0.62      0.61       349
weighted avg       0.63      0.61      0.62       349

0.6131805157593123


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Predicted,0,1
Actual,,
0,99,47
1,88,115
